In [2]:
%matplotlib inline
import json
import os
import hashlib
import pandas as pd
import matplotlib.pyplot as plt
from web3 import Web3
import datetime
import plotly.express as px


In [ ]:
# File paths
LOGS_DIR = "../logs/decisions"
ABI_PATH = "../blockchain/abi/DecisionLogger.json"
CONTRACT_ADDRESS =  # 🔁 Replace
GANACHE_URL = "http://127.0.0.1:7545"

# Load Web3 and contract
web3 = Web3(Web3.HTTPProvider(GANACHE_URL))
with open(ABI_PATH) as f:
    abi = json.load(f)
contract = web3.eth.contract(address=CONTRACT_ADDRESS, abi=abi)


In [ ]:
# Get latest log
log_files = sorted([f for f in os.listdir(LOGS_DIR) if f.endswith(".json")], reverse=True)
latest_log = os.path.join(LOGS_DIR, log_files[0])

with open(latest_log) as f:
    decision_log = json.load(f)

print("📄 Loaded Log ID:", decision_log["log_id"])
0

IndexError: list index out of range

In [5]:
input_df = pd.DataFrame(decision_log["input_features"].items(), columns=["Feature", "Value"])
input_df.set_index("Feature").T


Feature,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
Value,42.0,2519.0,15.0,262.0,4.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
import pandas as pd
import plotly.graph_objects as go

# Prepare SHAP data
shap_dict = decision_log["shap_contributions"]
input_dict = decision_log["input_features"]

shap_df = pd.DataFrame({
    "Feature": list(shap_dict.keys()),
    "SHAP Value": list(shap_dict.values())
})

# Add actual input values
shap_df["Input Value"] = shap_df["Feature"].map(input_dict)

# Sort by absolute SHAP impact and take top 10
shap_df["|SHAP|"] = shap_df["SHAP Value"].abs()
shap_df = shap_df.sort_values("|SHAP|", ascending=False).head(10)

# Format value label: show SHAP + input value
shap_df["Label"] = shap_df.apply(
    lambda row: f"Value: {row['Input Value']}<br>SHAP: {row['SHAP Value']:.3f}",
    axis=1
)

# Build chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=shap_df["Feature"],
    x=shap_df["SHAP Value"],
    orientation='h',
    marker=dict(
        color=shap_df["SHAP Value"],
        colorscale='RdBu',
        colorbar=dict(title='SHAP Value')
    ),
    hovertext=shap_df["Label"],
    hoverinfo="text"
))

# Add annotations for input values directly on the bars (optional)
fig.update_traces(text=shap_df["Input Value"], textposition='outside')

# Add decision details to title
log_id = decision_log.get("log_id", "N/A")
prediction = decision_log.get("prediction", None)
confidence = float(decision_log.get("probability", 0))

decision_text = f"✅ Approved" if prediction == 1 else "❌ Denied"

fig.update_layout(
    title=f"Top 10 SHAP Feature Contributions for Input ID: {log_id} ({decision_text}, Confidence: {confidence:})",
    xaxis_title="SHAP Value → Direction & Impact on Prediction",
    yaxis_title="Feature",
    height=550,
    margin=dict(t=100),
    bargap=0.4
)

fig.show()
